Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [43]:
batch_size = 128
# regularization = 0.1    # Test accuracy: 84.4%
# regularization = 0.01   # Test accuracy: 90.3%
# regularization = 0.005  # Test accuracy: 91.7%
regularization = 0.001    # Test accuracy: 93.3% <-- Best Result
# regularization = 0.0005 # Test accuracy: 91.4%
# regularization = 0.0001 # Test accuracy: 90.0%

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(hidden, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    regularization * tf.nn.l2_loss(weights1) + regularization * tf.nn.l2_loss(biases1) + 
    regularization * tf.nn.l2_loss(weights2) + regularization * tf.nn.l2_loss(biases2))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(
      tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(
      tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 626.542603
Minibatch accuracy: 4.7%
Validation accuracy: 33.8%
Minibatch loss at step 500: 205.423294
Minibatch accuracy: 80.5%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 116.752373
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 69.246758
Minibatch accuracy: 88.3%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 41.662926
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 25.537640
Minibatch accuracy: 85.9%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 15.552917
Minibatch accuracy: 85.9%
Validation accuracy: 86.3%
Test accuracy: 93.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [44]:
num_steps = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 665.241943
Minibatch accuracy: 6.2%
Validation accuracy: 31.9%
Test accuracy: 62.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [45]:
batch_size = 128
regularization = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  keep_prob = tf.placeholder(tf.float32)

  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden_drop = tf.nn.dropout(hidden, keep_prob)
  logits = tf.matmul(hidden_drop, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    regularization * tf.nn.l2_loss(weights1) + regularization * tf.nn.l2_loss(biases1) + 
    regularization * tf.nn.l2_loss(weights2) + regularization * tf.nn.l2_loss(biases2))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(
      tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(
      tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [46]:
num_steps = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 780.810425
Minibatch accuracy: 11.7%
Validation accuracy: 26.1%
Test accuracy: 56.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [178]:
batch_size = 128
regularization = 0.001

# Input nodes: 784
hidden_nodes1 = 1024
hidden_nodes2 = 512
hidden_nodes3 = 1024
# Output nodes: 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  keep_prob = tf.placeholder(tf.float32)

  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes1], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    
  weights3 = tf.Variable(
    tf.truncated_normal([hidden_nodes2, hidden_nodes3], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([hidden_nodes3]))
    
  weights4 = tf.Variable(
    tf.truncated_normal([hidden_nodes3, num_labels], stddev=0.1))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden_drop1 = tf.nn.dropout(hidden1, keep_prob)
  
  hidden2 = tf.nn.relu(tf.matmul(hidden_drop1, weights2) + biases2)
  hidden_drop2 = tf.nn.dropout(hidden2, keep_prob)
  
  hidden3 = tf.nn.relu(tf.matmul(hidden_drop2, weights3) + biases3)
  hidden_drop3 = tf.nn.dropout(hidden3, keep_prob)
  
  logits = tf.matmul(hidden_drop3, weights4) + biases4
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    regularization * tf.nn.l2_loss(weights1) + regularization * tf.nn.l2_loss(biases1) + 
    regularization * tf.nn.l2_loss(weights2) + regularization * tf.nn.l2_loss(biases2) + 
    regularization * tf.nn.l2_loss(weights3) + regularization * tf.nn.l2_loss(biases3) + 
    regularization * tf.nn.l2_loss(weights4) + regularization * tf.nn.l2_loss(biases4))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)
    
  # Predictions for the training data.
  train_prediction = tf.nn.softmax(logits)

  # Predictions for the validation data.
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(
      tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(
      tf_valid_dataset, weights1) + biases1), weights2) + 
      biases2), weights3) + biases3), weights4) + biases4)

  # Predictions for the test data.
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(
      tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(
      tf_test_dataset, weights1) + biases1), weights2) + 
      biases2), weights3) + biases3), weights4) + biases4)

In [179]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 37.719837
Minibatch accuracy: 8.6%
Validation accuracy: 10.9%
Minibatch loss at step 2000: 5.271484
Minibatch accuracy: 88.3%
Validation accuracy: 84.2%
Minibatch loss at step 4000: 3.673059
Minibatch accuracy: 86.7%
Validation accuracy: 85.8%
Minibatch loss at step 6000: 2.748472
Minibatch accuracy: 80.5%
Validation accuracy: 86.8%
Minibatch loss at step 8000: 2.189904
Minibatch accuracy: 78.9%
Validation accuracy: 87.5%
Minibatch loss at step 10000: 1.483261
Minibatch accuracy: 84.4%
Validation accuracy: 87.9%
Minibatch loss at step 12000: 1.222859
Minibatch accuracy: 83.6%
Validation accuracy: 88.3%
Minibatch loss at step 14000: 0.938541
Minibatch accuracy: 88.3%
Validation accuracy: 88.9%
Minibatch loss at step 16000: 0.697938
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 18000: 0.613568
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 20000: 0.709759
Minibatch accuracy: 85.2%
Validation 

### Results for various combinations of layer sizes

32, 64, 1024    with 20001 steps --> Test accuracy: 90.6%


64, 128, 1024    with 3001 steps --> Test accuracy: 90.5%

64, 128, 1024   with 20001 steps --> Test accuracy: 91.9%


256, 512, 1024  with 20001 steps --> Test accuracy: 94.7%


256, 32, 512    with 20001 steps --> Test accuracy: 93.4%


512, 64, 512     with 3001 steps --> Test accuracy: 91.5%

512, 64, 512    with 20001 steps --> Test accuracy: 94.4%


128, 1024, 128  with 20001 steps --> Test accuracy: 93.5%


512, 1024, 512  with 20001 steps --> Test accuracy: 94.9%


1024, 256, 1024  with 3001 steps --> Test accuracy: 90.6%

1024, 256, 1024 with 20001 steps --> Test accuracy: 95.2% <-- Highest Accuracy


1024, 512, 1024 with 20001 steps --> Test accuracy: 95.1% <-- Second Highest


1024, 1024, 1024 with 3001 steps --> Test accuracy: 91.0%

1024, 128, 4096 with 20001 steps --> Test accuracy: 94.9%